# Postflop Equity Analysis

Analyze equity on different board textures and see how draws affect equity.

In [ ]:
from holdem_lab import (
    parse_cards, format_cards,
    EquityRequest, PlayerHand, calculate_equity,
    evaluate_hand,
)
import matplotlib.pyplot as plt

## Set vs Flush Draw

In [ ]:
# Set vs flush draw on the flop
hole1 = parse_cards("Qh Qd")  # Set of Queens
hole2 = parse_cards("Ah Kh")  # Nut flush draw
board = parse_cards("Qs 7h 2h")  # Queen on board, two hearts

print(f"Player 1: {format_cards(hole1)} (Set of Queens)")
print(f"Player 2: {format_cards(hole2)} (Nut Flush Draw)")
print(f"Board: {format_cards(board)}")
print()

# Current hand evaluation
print("Current hands:")
print(f"  P1: {evaluate_hand(list(hole1) + list(board)).describe()}")
print(f"  P2: {evaluate_hand(list(hole2) + list(board)).describe()}")
print()

# Calculate equity
request = EquityRequest(
    players=[
        PlayerHand(hole_cards=tuple(hole1)),
        PlayerHand(hole_cards=tuple(hole2)),
    ],
    board=list(board),
    num_simulations=10000,
    seed=42,
)
result = calculate_equity(request)

print("Equity:")
print(f"  Set: {result.players[0].equity:.1%}")
print(f"  Flush Draw: {result.players[1].equity:.1%}")

## Made Hand vs Combo Draw

In [ ]:
# Overpair vs combo draw (flush + straight draw)
hole1 = parse_cards("Ah Ad")  # Overpair
hole2 = parse_cards("Jh Th")  # Combo draw
board = parse_cards("9h 8c 2h")  # Gives flush draw + open-ended straight draw

print(f"Player 1: {format_cards(hole1)} (Overpair)")
print(f"Player 2: {format_cards(hole2)} (Combo Draw)")
print(f"Board: {format_cards(board)}")
print()

# Calculate equity
request = EquityRequest(
    players=[
        PlayerHand(hole_cards=tuple(hole1)),
        PlayerHand(hole_cards=tuple(hole2)),
    ],
    board=list(board),
    num_simulations=10000,
    seed=42,
)
result = calculate_equity(request)

print("Equity:")
print(f"  Overpair: {result.players[0].equity:.1%}")
print(f"  Combo Draw: {result.players[1].equity:.1%}")
print("\n(Combo draws can have surprisingly high equity!)")

## Equity Through Streets

See how equity changes from flop to turn to river.

In [ ]:
# Track equity through streets
hole1 = parse_cards("Kh Kd")  # Overpair
hole2 = parse_cards("Ah Qh")  # Flush draw
flop = parse_cards("Jh 7h 2c")
turn = parse_cards("4s")
river = parse_cards("3d")

streets = [
    ("Flop", list(flop)),
    ("Turn", list(flop) + list(turn)),
    ("River", list(flop) + list(turn) + list(river)),
]

print(f"Hero: {format_cards(hole1)} (Kings)")
print(f"Villain: {format_cards(hole2)} (Flush Draw)")
print()

equities = []
for street_name, board in streets:
    print(f"{street_name}: {format_cards(board)}")
    
    request = EquityRequest(
        players=[
            PlayerHand(hole_cards=tuple(hole1)),
            PlayerHand(hole_cards=tuple(hole2)),
        ],
        board=board,
        num_simulations=10000,
        seed=42,
    )
    result = calculate_equity(request)
    eq1 = result.players[0].equity
    eq2 = result.players[1].equity
    equities.append((street_name, eq1, eq2))
    
    hand1 = evaluate_hand(list(hole1) + board)
    hand2 = evaluate_hand(list(hole2) + board)
    print(f"  KK: {eq1:.1%} ({hand1.describe()})")
    print(f"  AQh: {eq2:.1%} ({hand2.describe()})")
    print()

In [ ]:
# Plot equity through streets
fig, ax = plt.subplots(figsize=(8, 5))

street_names = [e[0] for e in equities]
eq1 = [e[1] * 100 for e in equities]
eq2 = [e[2] * 100 for e in equities]

x = range(len(street_names))
ax.plot(x, eq1, 'o-', label='KK', color='steelblue', linewidth=2, markersize=10)
ax.plot(x, eq2, 'o-', label='AQh', color='coral', linewidth=2, markersize=10)

ax.set_xticks(x)
ax.set_xticklabels(street_names)
ax.set_ylabel('Equity (%)')
ax.set_title('Equity Changes Through Streets')
ax.legend()
ax.set_ylim(0, 100)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Board Texture Analysis

In [ ]:
# Same hands, different board textures
hole1 = parse_cards("Ah Ad")  # Aces
hole2 = parse_cards("7c 7d")  # Sevens

boards = [
    ("Dry board", "Kc 9s 2h"),
    ("Paired board", "Kc Ks 2h"),
    ("Wet board (flush)", "8h 5h 2h"),
    ("Connected board", "Jh Ts 9c"),
    ("Set for 77", "7h Kc 2s"),
]

print(f"AA vs 77 on Different Board Textures")
print("=" * 50)

results = []
for name, board_str in boards:
    board = parse_cards(board_str)
    
    request = EquityRequest(
        players=[
            PlayerHand(hole_cards=tuple(hole1)),
            PlayerHand(hole_cards=tuple(hole2)),
        ],
        board=list(board),
        num_simulations=10000,
        seed=42,
    )
    result = calculate_equity(request)
    eq1 = result.players[0].equity
    eq2 = result.players[1].equity
    results.append((name, eq1, eq2))
    
    print(f"{name:20} | {board_str:15} | AA: {eq1:.1%} | 77: {eq2:.1%}")

In [ ]:
# Visualize board texture impact
fig, ax = plt.subplots(figsize=(10, 6))

names = [r[0] for r in results]
eq_aa = [r[1] * 100 for r in results]
eq_77 = [r[2] * 100 for r in results]

x = range(len(names))
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], eq_aa, width, label='AA', color='steelblue')
bars2 = ax.bar([i + width/2 for i in x], eq_77, width, label='77', color='coral')

ax.set_ylabel('Equity (%)')
ax.set_title('AA vs 77: Impact of Board Texture')
ax.set_xticks(x)
ax.set_xticklabels(names, rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 100)

plt.tight_layout()
plt.show()